--- Day 14: Regolith Reservoir ---

The distress signal leads you to a giant waterfall! Actually, hang on - the signal seems like it's coming from the waterfall itself, and that doesn't make any sense. However, you do notice a little path that leads behind the waterfall.

Correction: the distress signal leads you behind a giant waterfall! There seems to be a large cave system here, and the signal definitely leads further inside.

As you begin to make your way deeper underground, you feel the ground rumble for a moment. Sand begins pouring into the cave! If you don't quickly figure out where the sand is going, you could quickly become trapped!

Fortunately, your familiarity with analyzing the path of falling material will come in handy here. You scan a two-dimensional vertical slice of the cave above you (your puzzle input) and discover that it is mostly air with structures made of rock.

Your scan traces the path of each solid rock structure and reports the x,y coordinates that form the shape of the path, where x represents distance to the right and y represents distance down. Each path appears as a single line of text in your scan. After the first point of each path, each point indicates the end of a straight horizontal or vertical line to be drawn from the previous point. For example:
```
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
```
This scan means that there are two paths of rock; the first path consists of two straight lines, and the second path consists of three straight lines. (Specifically, the first path consists of a line of rock from 498,4 through 498,6 and another line of rock from 498,6 through 496,6.)

The sand is pouring into the cave from point 500,0.

Drawing rock as #, air as ., and the source of the sand as +, this becomes:
```

  4     5  5
  9     0  0
  4     0  3
0 ......+...
1 ..........
2 ..........
3 ..........
4 ....#...##
5 ....#...#.
6 ..###...#.
7 ........#.
8 ........#.
9 #########.
```
Sand is produced one unit at a time, and the next unit of sand is not produced until the previous unit of sand comes to rest. A unit of sand is large enough to fill one tile of air in your scan.

A unit of sand always falls down one step if possible. If the tile immediately below is blocked (by rock or sand), the unit of sand attempts to instead move diagonally one step down and to the left. If that tile is blocked, the unit of sand attempts to instead move diagonally one step down and to the right. Sand keeps moving as long as it is able to do so, at each step trying to move down, then down-left, then down-right. If all three possible destinations are blocked, the unit of sand comes to rest and no longer moves, at which point the next unit of sand is created back at the source.

So, drawing sand that has come to rest as o, the first unit of sand simply falls straight down and then stops:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.
```
The second unit of sand then falls straight down, lands on the first one, and then comes to rest to its left:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.
```
After a total of five units of sand have come to rest, they form this pattern:
```
......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.
```
After a total of 22 units of sand:
```
......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.
```
Finally, only two more units of sand can possibly come to rest:
```
......+...
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.
```
Once all 24 units of sand shown above have come to rest, all further sand flows out the bottom, falling into the endless void. Just for fun, the path any new sand takes before falling forever is shown here with ~:
```
.......+...
.......~...
......~o...
.....~ooo..
....~#ooo##
...~o#ooo#.
..~###ooo#.
..~..oooo#.
.~o.ooooo#.
~#########.
~..........
~..........
~..........
```
Using your scan, simulate the falling sand. How many units of sand come to rest before sand starts flowing into the abyss below?


In [1]:
// Data types and Functions
public record struct Position {
    public int X;
    public int Y;

    public static Position operator+(Position position, Position other) {
        return new Position {
            X = position.X + other.X,
            Y = position.Y + other.Y,
        };
    }
    
    public static Position operator-(Position position, Position other) {
        return new Position {
            X = position.X - other.X,
            Y = position.Y - other.Y,
        };
    }
}

Position ParsePosition(string input) {
    var parts = input.Split(",");
    return new Position {
        X = int.Parse(parts[0]),
        Y = int.Parse(parts[1]),
    };
}

IEnumerable<Position> PointsBetween(Position pos1, Position pos2) {
    var xValues = Enumerable.Range(Math.Min(pos1.X, pos2.X), Math.Abs(pos2.X - pos1.X) + 1);
    var yValues = Enumerable.Range(Math.Min(pos1.Y, pos2.Y), Math.Abs(pos2.Y - pos1.Y) + 1);
    foreach (var x in xValues) {
        foreach (var y in yValues) {
            yield return new Position {
                X = x,
                Y = y,
            };
        }
    }
}

public record struct Bounds {
    public int MinX;
    public int MaxX;
    public int MinY;
    public int MaxY;

    public bool ContainsPoint(Position point) {
        return point.X >= MinX
            && point.X <= MaxX
            && point.Y >= MinY
            && point.Y <= MaxY;
    }
}

Bounds GetBounds(IEnumerable<Position> rocks) {
    return new Bounds {
        MinX = rocks.MinBy(rock => rock.X).X,
        MinY = rocks.MinBy(rock => rock.Y).Y,
        MaxX = rocks.MaxBy(rock => rock.X).X,
        MaxY = rocks.MaxBy(rock => rock.Y).Y,
    };
}

public record struct State {
    public HashSet<Position> Rocks;
    public HashSet<Position> Sand;

    public bool IsEmpty(Position p) {
        return !Rocks.Contains(p) && !Sand.Contains(p);
    }
}

string DrawMap(State state) {
    var bounds = GetBounds(state.Rocks.Union(state.Sand));
    var xValues = Enumerable.Range(bounds.MinX, (bounds.MaxX - bounds.MinX) + 1);
    var yValues = Enumerable.Range(bounds.MinY, (bounds.MaxY - bounds.MinY) + 1);
    var buffer = new StringBuilder();
    foreach (var y in yValues) {
        foreach (var x in xValues) {
            if (state.Rocks.Contains(new Position { X = x, Y = y})) {
                buffer.Append("#");
            } else if (state.Sand.Contains(new Position { X = x, Y = y})) {
                buffer.Append("O");
            } else {
                buffer.Append(".");
            }
        }
        buffer.Append("\n");
    }
    return buffer.ToString();
}

public record struct SpawnResult {
    public State NewState;
    public bool SandFellIntoAbyss;
}

public SpawnResult SpawnSand(State previous, Bounds bounds) {
    var sandPosition = new Position { X = 500, Y = 0 };
    while (sandPosition.Y <= bounds.MaxY) {
        var candidates = new List<Position> {
            sandPosition + new Position { X = 0, Y = 1 },
            sandPosition + new Position { X = -1, Y = 1 },
            sandPosition + new Position { X = 1, Y = 1 },
        }.Where(candidate => previous.IsEmpty(candidate));
        if (candidates.Any()) {
            sandPosition = candidates.First();
        } else {
            previous.Sand.Add(sandPosition);
            return new SpawnResult { NewState = previous, SandFellIntoAbyss = false};
        }
    }
    return new SpawnResult { NewState = previous, SandFellIntoAbyss = true};
}

public record struct SpawnResultPart2 {
    public State NewState;
    public bool SandCouldNotMove;
}

public SpawnResultPart2 SpawnSandPart2(State previous, Bounds bounds) {
    var sandPosition = new Position { X = 500, Y = 0 };
    while (sandPosition.Y < bounds.MaxY + 1) {
        var candidates = new List<Position> {
            sandPosition + new Position { X = 0, Y = 1 },
            sandPosition + new Position { X = -1, Y = 1 },
            sandPosition + new Position { X = 1, Y = 1 },
        }.Where(candidate => previous.IsEmpty(candidate));
        if (candidates.Any()) {
            sandPosition = candidates.First();
        } else {
            break;
        }
    }
    previous.Sand.Add(sandPosition);
    return new SpawnResultPart2 {
        NewState = previous,
        SandCouldNotMove = sandPosition == new Position { X = 500, Y = 0 },
    };
}

The below script needs to be able to find the current output cell; this is an easy method to get it.


(66,30): warning CS0649: Field 'State.Rocks' is never assigned to, and will always have its default value null

(67,30): warning CS0649: Field 'State.Sand' is never assigned to, and will always have its default value null



In [2]:
// Read input
var input = System.IO.File.ReadAllLines("example.txt");
input.Take(10)

index,value
0,"498,4 -> 498,6 -> 496,6"
1,"503,4 -> 502,4 -> 502,9 -> 494,9"


In [3]:
// Part 1
var rocks = input.Select(line => line.Split(" -> ").Select(ParsePosition))
    .SelectMany(lineGroup => lineGroup.Zip(lineGroup.Skip(1)))
    .SelectMany(line => PointsBetween(line.First, line.Second))
    .ToHashSet();

var state = new State {
    Rocks = rocks,
    Sand =  new HashSet<Position>(),
};

var keepSpawning = true;
var sandCount = 0;
var bounds = GetBounds(state.Rocks);
while (keepSpawning) {
    var result = SpawnSand(state, bounds);
    state = result.NewState;
    if (result.SandFellIntoAbyss) {
        keepSpawning = false;
    } else {
        sandCount += 1;
    }
};
Console.WriteLine(DrawMap(state));
sandCount


......O...
.....OOO..
....#OOO##
...O#OOO#.
..###OOO#.
....OOOO#.
.O.OOOOO#.
#########.



24

In [4]:
// Part 2
var rocks = input.Select(line => line.Split(" -> ").Select(ParsePosition))
    .SelectMany(lineGroup => lineGroup.Zip(lineGroup.Skip(1)))
    .SelectMany(line => PointsBetween(line.First, line.Second))
    .ToHashSet();

var state = new State {
    Rocks = rocks,
    Sand =  new HashSet<Position>(),
};

var keepSpawning = true;
var sandCount = 0;
var bounds = GetBounds(state.Rocks);
while (keepSpawning) {
    var result = SpawnSandPart2(state, bounds);
    state = result.NewState;
    sandCount += 1;
    if (result.SandCouldNotMove) {
        keepSpawning = false;
    }
}
Console.WriteLine(DrawMap(state));
sandCount

..........O..........
.........OOO.........
........OOOOO........
.......OOOOOOO.......
......OO#OOO##O......
.....OOO#OOO#OOO.....
....OO###OOO#OOOO....
...OOOO.OOOO#OOOOO...
..OOOOOOOOOO#OOOOOO..
.OOO#########OOOOOOO.
OOOOO.......OOOOOOOOO



93